<a href="https://colab.research.google.com/github/IvanOlvera25/2022-1-Comp-I/blob/main/SistemaNotificaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sistema de notificaciones

In [ ]:
import getpass
print("Para acceder favor de ingresar tus datos correctamente (tienes 3 intentos)")
contador = 1
while contador <=3:
  usuario = input("Ingresa tu nombre de usuario: ")
  contraseña = getpass.getpass("Ingresa tu contraseña: ")
  if usuario == "ivan"  and contraseña == "grocery":
    print("Bienvenid@ al sistema de notificaciones de 'Grocery'")
    #Importamos todas las librerías necesarias
    import pandas as pd
    from google.colab import files
    import time
    from datetime import datetime
    from datetime import timedelta
    import smtplib
    from email.mime.text import MIMEText  
    #Lo primero es subir y leer nuestro archivo csv
    print("Favor de elegir el inventario a usar")
    files.upload() 
    inv = pd.read_csv("/content/inventory.csv")
    #Declaramos una variable con la fecha del día de hoy
    x = time.strftime("%d/%m/%Y")
    #Con la librería datetime podemos hacer operaciones con las fechas, por lo que creamos una función que le sume 7 días a la fecha actual y le asignamos una variable
    Hoy = datetime.now()
    En_una_semana = Hoy + timedelta(days=7)
    y = En_una_semana.strftime("%d/%m/%Y")
    #Hacemos un subset de nombres y fecha de caducidad para poder hacer las funciones
    invN = inv.sort_values("Nombre")
    invC = inv.sort_values("Caducidad")
    #Creamos nuestra variable con el nombre y id de nuestro producto por caducar hoy
    co = invC[invC["Caducidad"] == x]
    ch = str(co[["Nombre","id"]])
    #Ahora volvemos a hacer  lo mismo para nuestro producto que caduca en una semana
    ce = invC[invC["Caducidad"] == y]
    cE = str(ce[["Nombre","id"]])
    #Ahora crearemos un subset con la suma de los valores repetidos en la columna Nombre
    reps = inv["Nombre"].value_counts()
    #Creamos el Dataframe para poder iterar por columnas
    rep2 = reps.rename_axis("Nombres")
    rep3 = rep2.reset_index()
    #Creamos nuestra variable con los productos con un inv menor a 2
    rp = rep3[rep3["Nombre"] < 2]
    rpn = str(rp["Nombres"])
    #Ahora accedemos al servidor 
    server=smtplib.SMTP('smtp.gmail.com',587)
    server.starttls()
    server.login(EMAIL,PASSWORD)
    #Pedimos al usuario a que correo le enviaremos las alertas
    TO = input("¿A quien le vamos a enviar el correo? ")
    #Creamos los mensajes o correos predeterminados para cada caso
    message=MIMEText("Hay poco inventario del producto:\n" + rpn)
    message["From"]=EMAIL
    message["To"]=TO
    message["Subject"]="Solo queda una unidad de este producto!"

    message3=MIMEText("Precaución! Hoy caduca:\n" + ch)
    message3["From"]=EMAIL
    message3["To"]=TO
    message3["Subject"]="Hay un producto a punto de caducar"

    message2=MIMEText("Este producto caduca en una semana:\n" + cE)
    message2["From"]=EMAIL
    message2["To"]=TO
    message2["Subject"]="Hay un producto que caduca en una semana!"

    #Ya teniendo los mensajes llamamos a la funciones de cada alerta
    AlertaHoy(inv)
    AlertaSemana(inv)
    AlertaPoco(inv)
    print("Las alertas ya han sido enviadas")
    break
  else:
    print("Los datos son incorrectos")
    if contador == 3:
      print("Comunicate con el administrador, has llegado al numero maximo de intentos")
    contador = contador +1

In [33]:
def AlertaHoy(inv):
  for i in invC["Caducidad"]:
    if i == x:
      server.sendmail(EMAIL,TO,message3.as_string())
  return
def AlertaSemana(inv):
  for i in invC["Caducidad"]:
    if i == y:
      server.sendmail(EMAIL,TO,message2.as_string())
  return
def AlertaPoco(inv):
  for i in rep3["Nombre"]:
    if i < 2:
      server.sendmail(EMAIL,TO,message.as_string())
  return 


In [6]:
PASSWORD = "Smtplib25@"
EMAIL = "compubot25@gmail.com"

In [ ]:
reps